# Training Against QM Energies

This notebook aims to show how the [`descent`](https://github.com/SimonBoothroyd/descent) framework in combination with
[`smirnoffee`](https://github.com/SimonBoothroyd/smirnoffee) can be used to train a set of SMIRNOFF force field bond and
angle force constant parameters against the QM derived relative energies of a small molecule in multiple conformers.

For the sake of clarity all warning will be disabled:

In [1]:
import warnings
warnings.filterwarnings('ignore')
import logging
logging.getLogger("openff.toolkit").setLevel(logging.ERROR)

### Retrieving the QM training set

For this example we will be training against QM energies which have been computed by and stored within the
[QCArchive](https://qcarchive.molssi.org/), which are easily retrieved using the [OpenFF QCSubmit](https://github.com/openforcefield/openff-qcsubmit)
package.

We begin by importing the records associated with the `OpenFF Optimization Set 1` optimization data set:

In [2]:
from qcportal import FractalClient

from openff.qcsubmit.results import OptimizationResultCollection

result_collection = OptimizationResultCollection.from_server(
    client=FractalClient(),
    datasets="OpenFF Optimization Set 1"
)

which we will then filter to retain a small molecule which will be fast to train on as a demonstration:

In [3]:
from openff.qcsubmit.results.filters import SMILESFilter

result_collection = result_collection.filter(
    SMILESFilter(smiles_to_include=["CC(=O)NCC1=NC=CN1C"])
)

print(f"N Molecules: {result_collection.n_molecules}")
print(f"N Molecules: {result_collection.n_results}")

N Molecules: 1
N Molecules: 6


You should see that our filtered collection contains the 6 results, which corresponds to 6 minimized conformers (and
their associated energy computed using the OpenFF default B3LYP-D3BJ spec) for the molecule we filtered for above.

### Defining the Objective ( / loss) function

For this example we will be training our force field parameters against the relative energies between the conformers we
retrieved above. In particular, we will use the following L2 loss function:

$$L2\left(\Delta\theta\right) = \dfrac{2}{N\left(N - 1 \right )}\sum ^N _{i=1} \sum ^N_{j = i+i} \left[ \left(E_{QM,i}-E_{QM,j}\right) - \left(E_{MM,i}\left(\theta + \Delta\theta \right) - E_{MM,j}\left(\theta + \Delta\theta \right) \right) \right ]^2$$

where N is the number of conformers of our given molecule, $E_{QM,i}$ the QM computed energy of conformer $i$,
$E_{MM,i}$ the MM energy of conformer $i$, $\theta$ the initial force field parameters, and $\Delta\theta$ the
delta term that is being trained and added to the original force field parameters.

This loss function is directly encoded into ``descent`` through the ``RelativeEnergyObjective`` object which can be
created directly from the collection of optimization results we retrieved above.

We first load in the initial force field parameters ($\theta$) using the [OpenFF Toolkit](https://github.com/openforcefield/openff-toolkit):

In [4]:
from openff.toolkit.typing.engines.smirnoff import ForceField
initial_force_field = ForceField("openff_unconstrained-1.0.0.offxml")

which we can then use to construct our contribution objects:

In [5]:
from descent.objectives.energy import RelativeEnergyObjective

objective_contributions = RelativeEnergyObjective.from_optimization_results(
    result_collection, initial_force_field, include_gradients=False
)

The returned `objective_contributions` will contain one objective object per unique molecule in the
`result_collection`:

In [6]:
len(objective_contributions)

1

as we filtered our initial result collection to only contain a single molecule, so too do we only have a single
contribution object.

### Specifying the parameters to train

For this example will will train all of the bond and force constants that were assigned to our molecule
of interest:

In [7]:
parameter_delta_ids = sorted(
    {
        (handler_type, potential_key, attribute)
        for contribution in objective_contributions
        for handler_type, potential_key, attribute in contribution.parameter_ids
        if attribute in "k" and handler_type in ["Bonds", "Angles"]
    },
    key=lambda x: x[0],
    reverse=True
)

where here we have made use of the ``RelativeEnergyObjective.parameter_ids`` which contains the unique identifiers of
the parameters that were assigned to the molecule referenced by the object.

In [8]:
parameter_delta_ids[:2]

[('Bonds', PotentialKey(id='[#6:1]=[#8X1+0,#8X2+1:2]', mult=None), 'k'),
 ('Bonds', PotentialKey(id='[#6X3:1]=[#6X3:2]', mult=None), 'k')]

These ids are comprised of the type of SMIRNOFF parameter handler that the parameter originated from,
a key containing the id (in this case the SMIRKS pattern) associated with the parameter and the specific
attribute of the parameter (e.g. the force constant ``k``).

These keys will allow us to map our tensor of delta values:

In [9]:
import torch

parameter_delta = torch.zeros(len(parameter_delta_ids), requires_grad=True)

easily back to more meaningful force field parameters.

### Training the force field parameters

We are finally ready to begin training our force field parameters, or more precisely, the delta value that
we should perturb the force field parameters by to reach better agreement with the training data.

Here we will use the 'boilerplate Pytorch optimization loop':

In [10]:
lr = 0.1
n_epochs = 200

optimizer = torch.optim.Adam([parameter_delta], lr=lr)

for epoch in range(n_epochs):

    loss = torch.zeros(1)

    for objective_contribution in objective_contributions:
        loss += objective_contribution(parameter_delta, parameter_delta_ids)

    loss.backward()

    optimizer.step()
    optimizer.zero_grad()

    if epoch % 20 == 0:
        print(f"Epoch {epoch}: loss={loss.item()}")

Epoch 0: loss=0.10203274339437485
Epoch 20: loss=0.037074923515319824
Epoch 40: loss=0.0014350797282531857
Epoch 60: loss=0.0023795373272150755
Epoch 80: loss=0.001053333398886025
Epoch 100: loss=0.0008100062841549516
Epoch 120: loss=0.0007415462168864906
Epoch 140: loss=0.0006738712545484304
Epoch 160: loss=0.0006102666957303882
Epoch 180: loss=0.0005511701456271112


where the only code of note is the loop over our objective contributions which get added to the total
loss function.

We can save our trained parameters back to a SMIRNOFF `.offxml` file for future use:

In [11]:
from descent.utilities.smirnoff import perturb_force_field

final_force_field = perturb_force_field(
    initial_force_field, parameter_delta, parameter_delta_ids
)
final_force_field.to_file("final.offxml")